In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('lixas111.csv')

In [ ]:
df_one_hot = pd.get_dummies(df, columns=['job','marital','education','default','housing','loan','contact','poutcome'])
df_one_hot

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,29,751,23,121,2,-1,0,no,0,0,...,1,1,0,0,0,1,0,0,0,1
1,47,6695,21,129,3,-1,0,no,1,0,...,0,1,0,0,1,0,0,0,0,1
2,26,82,5,228,1,-1,0,no,1,0,...,1,1,0,0,0,1,0,0,0,1
3,66,53,12,562,4,-1,0,yes,0,0,...,0,1,0,1,0,0,0,0,0,1
4,36,127,18,172,3,-1,0,no,0,0,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,47,4157,11,90,3,-1,0,no,0,1,...,0,1,0,0,0,1,0,0,0,1
3996,53,2124,17,121,2,344,2,no,0,0,...,1,1,0,1,0,0,1,0,0,0
3997,30,31,8,229,1,-1,0,no,0,0,...,0,1,0,0,0,1,0,0,0,1
3998,43,743,23,323,3,-1,0,no,0,0,...,0,0,1,1,0,0,0,0,0,1


In [ ]:
df_x = df_one_hot.drop('y',axis=1)
df_x = df_x.drop(['duration','pdays'],axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_y = df_one_hot['y']

df_y

le = LabelEncoder()


y = le.fit_transform(df_y)

In [ ]:
df_y = pd.DataFrame(y, columns=['y'])
df_y

,y
0,0
1,0
2,0
3,1
4,0
...,...
3995,0
3996,0
3997,0
3998,0


In [ ]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.30, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
preds = clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
confusion_matrix(y_test, preds)

array([[765, 166],
       [168, 101]])

In [ ]:
accuracy_score(y_test, preds)

0.7216666666666667

In [ ]:
Feature_Importance = pd.DataFrame(data=clf.feature_importances_, index=df_x.columns, columns=['Importance'])
Feature_Importance
import numpy as np

top_n_features = Feature_Importance.sort_values(by='Importance', axis=0, ascending=False)[:10]
top_n_features

,Importance
balance,0.207563
age,0.177478
day,0.139276
poutcome_success,0.112001
campaign,0.053896
previous,0.024868
marital_single,0.022644
education_secondary,0.021944
job_technician,0.020178
education_tertiary,0.020146


In [ ]:

# Prune the tree by setting the max_depth parameter to 3
clf_pruned = DecisionTreeClassifier(max_depth=5)
clf_pruned.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=5)

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
# Import necessary libraries
import graphviz
from sklearn import tree

# Create a graphviz object
# Export the decision tree using only the top n important features
dot_data = tree.export_graphviz(clf_pruned, out_file=None, 
                                feature_names=df_x.columns,  
                                class_names=le.classes_,  
                                filled=True, rounded=True,  
                                special_characters=True)

# Create a graph from the dot data
graph = graphviz.Source(dot_data)

# Visualize the graph
graph.render("decision_tree")


'decision_tree.pdf'

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# Define the KFold cross-validator
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

# Use the cross_val_score function to evaluate the model using K-fold cross-validation
scores = cross_val_score(clf, df_x, df_y, cv=kfold)

# Print the mean and standard deviation of the scores
print("Accuracy: %.2f%% (%.2f%%)" % (scores.mean()*100, scores.std()*100))

Accuracy: 72.47% (0.78%)
